In [1]:
import orion_recommend
from orion_recommend.models import HybridMatrixFactorizationModel as HMFModel
from orion_recommend.models import MatrixFactorizationModel as MFModel
from scipy import sparse as sp
import pandas as pd
import numpy as np
from orion_recommend.datasets import DatasetMap, synthetic, utils, Encoder
from orion.sources import S3Source
from orion.sources.io import read_csv, write_csv
from orion.sources import RedshiftSource
import pandas as pd
import numpy as np
from scipy import sparse
from orion_recommend.evaluate import metrics
import implicit
import lightfm
import evall

In [2]:
def metrics_user_rec(item, sim_users, k):
    count = 0
    for i, user_list in zip(item, sim_users):       
        for u in user_list:
            if ui_matrix[u,i] == 1:
                count = count + 1            
    p_at_k = round(count/(item.shape[0] * k), 4)

    RS = []
    ans = 0.0
    for i, user_list in zip(item, sim_users):           
        r=[]
        for u in user_list:
             r.append(ui_matrix[u][i])
        ans = ans + evall.ndcg_at_k(r, k, method=1)
        RS.append(r)
    G_at_k = ans/item.shape[0]
    M_at_k = evall.mean_average_precision(RS)
    return p_at_k, G_at_k, M_at_k

def metrics_item_rec(user, sim_items, k):
    count = 0
    for u, item_list in zip(user, sim_items):       
        for i in item_list:
            if ui_matrix[u,i] == 1:
                count = count + 1            
    p_at_k = round(count/(user.shape[0] * k), 4)

    RS = []
    ans = 0.0
    for u, item_list in zip(user, sim_items):           
        r=[]
        for i in item_list:
             r.append(ui_matrix[u][i])
        ans = ans + evall.ndcg_at_k(r, k, method=1)
        RS.append(r)
    G_at_k = ans/user.shape[0]
    M_at_k = evall.mean_average_precision(RS)
    return p_at_k, G_at_k, M_at_k


### Orion baselines

In [3]:
ui_matrix= np.load("fa_ui_matrix.npy")

In [4]:
users = pd.read_csv("users", index_col= 0)
items = pd.read_csv("items", index_col= 0)
interactions = pd.read_csv("interactions", index_col= 0)

In [5]:
users.user_id = users.user_id.astype("int32")

In [6]:
items.item_id = items.item_id.astype("int32")

In [7]:
u = sp.csr_matrix(users)
i = sp.csr_matrix(items)
ints = sp.csr_matrix(interactions)

In [8]:
train, test = DatasetMap.split(interactions, users=users, items=items, frac_train=0.8, validate_inputs = True)

/home/declanbarrycbc1/thesis/src/orion-recommend/orion_recommend/evaluate/utils.py:176: UserWarning: The total number of users that can be sampled from for your test set is less than 75% of the available users. Consider reducing yourchosen 'n_test' (it is set to 1).
  warnings.warn(
/home/declanbarrycbc1/thesis/src/orion-recommend/orion_recommend/datasets/dataset_map.py:222: UserWarning: There are elements in 'customer_list' that are not in 'users'.
  warnings.warn(
/home/declanbarrycbc1/thesis/src/orion-recommend/orion_recommend/datasets/dataset_map.py:222: UserWarning: There are elements in 'inventory' that are not in 'items'.
  warnings.warn(
/home/declanbarrycbc1/thesis/src/orion-recommend/orion_recommend/datasets/dataset_map.py:222: UserWarning: There are elements in 'customer_list' that are not in 'users'.
  warnings.warn(
/home/declanbarrycbc1/thesis/src/orion-recommend/orion_recommend/datasets/dataset_map.py:222: UserWarning: There are elements in 'inventory' that are not in 'i

In [9]:
train.encoded_users.shape

(86284,)

In [10]:
opp, opp1 = DatasetMap.split(interactions.iloc[:,[1,0]], users=items, items=users, frac_train=0.8, validate_inputs = True)

/home/declanbarrycbc1/thesis/src/orion-recommend/orion_recommend/datasets/dataset_map.py:222: UserWarning: There are elements in 'inventory' that are not in 'items'.
  warnings.warn(
/home/declanbarrycbc1/thesis/src/orion-recommend/orion_recommend/datasets/dataset_map.py:222: UserWarning: There are elements in 'customer_list' that are not in 'users'.
  warnings.warn(
/home/declanbarrycbc1/thesis/src/orion-recommend/orion_recommend/datasets/dataset_map.py:222: UserWarning: There are elements in 'inventory' that are not in 'items'.
  warnings.warn(


In [11]:
hmod = HMFModel()

In [12]:
model = MFModel()

In [13]:
hmod.fit(train)

In [14]:
model.fit(train)

  0%|          | 0/15 [00:00<?, ?it/s]

In [15]:
%%capture
recommended_items_bpr = model.recommend(test.users, n=10)

In [16]:
preds_10 = model.predict(test.items,n=10)
preds_20 =  model.predict(test.items,n=20)

Recommending Chunk 0: 100%|██████████| 5189/5189 [00:05<00:00, 897.45it/s]

Recommending Chunk 0: 100%|██████████| 5189/5189 [00:06<00:00, 817.26it/s]

Recommending Chunk 1: 100%|██████████| 5188/5188 [00:06<00:00, 761.24it/s]


In [17]:
preds_10_arr = np.array(list(preds_10.values()))

In [18]:
metrics.precision_at_k(test.items,preds_10_arr)

/home/declanbarrycbc1/thesis/src/orion-recommend/orion_recommend/evaluate/metrics.py:151: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if element in actual:


0.0

In [12]:
print(hmod.score(test,train, k=10, fn=metrics.ndcg),
model.score(test, train, k=10, fn=metrics.precision_at_k),
     model.score(test,train, k=20, fn=metrics.ndcg),
model.score(test, train, k=20, fn=metrics.precision_at_k))

TypeError: Invalid type passed to user_ids parameter. This must be either int or np.int32 array. Type received: <class 'numpy.int64'>

In [13]:
print(model.score(test,train, k=10, fn=metrics.ndcg),
model.score(test, train, k=10, fn=metrics.precision_at_k),
     model.score(test,train, k=20, fn=metrics.ndcg),
model.score(test, train, k=20, fn=metrics.precision_at_k))

0.03450249835621144 0.2907692307692308 0.03306735631791807 0.2907692307692308


### Flip

In [7]:
model.fit(opp)

  0%|          | 0/15 [00:00<?, ?it/s]

In [63]:
recs = model.recommend(opp.users, chunk_size = 20000,n=10)


Recommending Chunk 0: 100%|██████████| 10377/10377 [00:20<00:00, 506.83it/s]


In [20]:
recs = recommended_items_bpr

In [21]:
np.array([*recs.values()])
sim_users = np.array([*recs.values()])
sim_users = [list(map(int, sim_users[i])) for i in range(sim_users.shape[0])]

In [26]:
np.array(sim_users).shape

(86284, 10)

In [23]:
metrics_item_rec(test.users, sim_users, 10)

(0.0005, 0.002309164708751765, 0.0014138663902814846)

In [11]:
print(model.score(opp1,opp, k=10, fn=metrics.ndcg),
model.score(opp1, opp, k=10, fn=metrics.precision_at_k),
     model.score(opp1,opp, k=20, fn=metrics.ndcg),
model.score(opp1, opp, k=20, fn=metrics.precision_at_k))

0.0012423852729491474 0.012180974477958236 0.0013672086866157247 0.012180974477958236
